# 4차시: 대화 상태 관리 & 멀티턴 최적화

## 학습 목표
- Redis 기반 세션 관리 이해
- 컨텍스트 윈도우 최적화 구현
- 대화 상태 추적 및 흐름 제어
- 멀티 사용자 환경에서의 동시성 처리

## 주요 구현 내용
1. **세션 관리 시스템**: Redis 기반 대화 히스토리 저장
2. **컨텍스트 최적화**: 토큰 수 기반 메시지 압축
3. **대화 흐름 제어**: 의도 분류 및 상태 전환
4. **멀티턴 최적화**: 중요도 기반 메시지 선별

## 1. 환경 설정 및 라이브러리 import

In [ ]:
#!/usr/bin/env python3
import os
import sys
import json
import time
import uuid
import logging
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from enum import Enum

# 외부 라이브러리
import redis
from openai import OpenAI
import tiktoken

# 로컬 설정
sys.path.append('..')
from config import get_config

config = get_config()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 환경 설정 완료")
print(f"📊 설정된 모델: {config.llm.openai_model}")
print(f"🔧 최대 토큰: {config.llm.max_tokens}")

## 2. 데이터 구조 정의

대화 관리를 위한 핵심 데이터 클래스들을 정의합니다.

In [ ]:
class ConversationState(Enum):
    """대화 상태 정의"""
    GREETING = "greeting"
    INFORMATION_SEEKING = "information_seeking"
    TASK_EXECUTION = "task_execution"
    PROBLEM_SOLVING = "problem_solving"
    CASUAL_CHAT = "casual_chat"
    ENDING = "ending"
    UNKNOWN = "unknown"

class MessageType(Enum):
    """메시지 유형"""
    USER = "user"
    ASSISTANT = "assistant"
    SYSTEM = "system"
    FUNCTION = "function"

@dataclass
class ConversationMessage:
    """대화 메시지 구조"""
    id: str
    role: MessageType
    content: str
    timestamp: datetime
    metadata: Dict[str, Any]
    tokens: int = 0
    importance_score: float = 0.0
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'id': self.id,
            'role': self.role.value,
            'content': self.content,
            'timestamp': self.timestamp.isoformat(),
            'metadata': self.metadata,
            'tokens': self.tokens,
            'importance_score': self.importance_score
        }

# 테스트: 메시지 객체 생성
test_message = ConversationMessage(
    id=str(uuid.uuid4()),
    role=MessageType.USER,
    content="안녕하세요! AI 챗봇에 대해 알고 싶습니다.",
    timestamp=datetime.now(),
    metadata={'source': 'test'}
)

print("✅ 데이터 구조 정의 완료")
print(f"📝 테스트 메시지 ID: {test_message.id[:8]}...")
print(f"🕐 생성 시간: {test_message.timestamp.strftime('%H:%M:%S')}")

## 3. 세션 관리 시스템 구현

Redis 기반으로 대화 세션을 관리하는 시스템을 구현합니다.

In [ ]:
class SessionManager:
    """Redis 기반 세션 관리자"""
    
    def __init__(self, redis_url: str = None, session_timeout: int = 3600):
        self.redis_url = redis_url or config.get_redis_url()
        self.session_timeout = session_timeout
        self.client = None
        self.fallback_sessions = {}  # Redis 실패 시 메모리 백업
        
        self._init_redis()
        logger.info(f"세션 매니저 초기화 완료 - 만료시간: {session_timeout}초")
    
    def _init_redis(self):
        """Redis 연결 초기화"""
        try:
            self.client = redis.from_url(
                self.redis_url,
                decode_responses=True,
                socket_connect_timeout=5
            )
            self.client.ping()
            logger.info("Redis 연결 성공")
        except Exception as e:
            logger.warning(f"Redis 연결 실패, 메모리 모드로 전환: {e}")
            self.client = None
    
    def create_session(self, user_id: str):
        """새 대화 세션 생성"""
        session_id = str(uuid.uuid4())
        now = datetime.now()
        
        session_data = {
            'session_id': session_id,
            'user_id': user_id,
            'created_at': now.isoformat(),
            'last_activity': now.isoformat(),
            'messages': [],
            'current_state': ConversationState.GREETING.value,
            'total_tokens': 0,
            'context_metadata': {
                'topic_history': [],
                'user_preferences': {}
            }
        }
        
        self._save_session_data(session_id, session_data)
        logger.info(f"새 세션 생성: {session_id} (사용자: {user_id})")
        
        return session_id, session_data
    
    def get_session(self, session_id: str) -> Optional[Dict]:
        """세션 조회"""
        try:
            if self.client:
                session_data = self.client.get(f"session:{session_id}")
                if session_data:
                    return json.loads(session_data)
            else:
                return self.fallback_sessions.get(session_id)
        except Exception as e:
            logger.error(f"세션 조회 실패: {e}")
        return None
    
    def _save_session_data(self, session_id: str, data: Dict):
        """세션 데이터 저장"""
        try:
            session_json = json.dumps(data, ensure_ascii=False)
            
            if self.client:
                self.client.setex(f"session:{session_id}", self.session_timeout, session_json)
            else:
                self.fallback_sessions[session_id] = data
        except Exception as e:
            logger.error(f"세션 저장 실패: {e}")
    
    def _count_tokens(self, text: str) -> int:
        """토큰 수 계산"""
        try:
            encoding = tiktoken.encoding_for_model("gpt-4")
            return len(encoding.encode(text))
        except:
            # 대략적 계산
            return len(text) // 4

# 세션 관리자 테스트
session_manager = SessionManager()
test_session_id, test_session = session_manager.create_session("test_user")

print("✅ 세션 관리 시스템 구현 완료")
print(f"🆔 테스트 세션 ID: {test_session_id[:8]}...")
print(f"👤 사용자 ID: {test_session['user_id']}")
print(f"🔄 현재 상태: {test_session['current_state']}")
print(f"🔗 Redis 연결: {'성공' if session_manager.client else '메모리 모드'}")

## 4. 컨텍스트 윈도우 최적화

토큰 제한을 고려한 컨텍스트 압축 및 최적화를 구현합니다.

In [ ]:
class ContextWindowOptimizer:
    """컨텍스트 윈도우 최적화"""
    
    def __init__(self, max_tokens: int = 4000, compression_ratio: float = 0.5):
        self.max_tokens = max_tokens
        self.compression_ratio = compression_ratio
        self.client = OpenAI(api_key=config.llm.openai_api_key)
        
        print(f"✅ 컨텍스트 최적화기 초기화 - 최대 토큰: {max_tokens}")
    
    def optimize_messages(self, messages: List[Dict], current_tokens: int) -> Tuple[List[Dict], Dict]:
        """메시지 최적화"""
        print(f"📊 컨텍스트 최적화 시작 - 현재 토큰: {current_tokens}")
        
        if current_tokens <= self.max_tokens:
            return messages, {'optimization': 'none', 'tokens': current_tokens}
        
        # 최근 메시지 우선 보존
        target_tokens = int(self.max_tokens * self.compression_ratio)
        optimized_messages = []
        token_count = 0
        
        # 시스템 메시지는 항상 포함
        system_messages = [msg for msg in messages if msg.get('role') == 'system']
        optimized_messages.extend(system_messages)
        token_count += sum(self._estimate_tokens(msg['content']) for msg in system_messages)
        
        # 최근 메시지부터 역순으로 추가
        recent_messages = [msg for msg in messages if msg.get('role') != 'system']
        for msg in reversed(recent_messages):
            msg_tokens = self._estimate_tokens(msg['content'])
            if token_count + msg_tokens <= target_tokens:
                optimized_messages.insert(-len(system_messages) if system_messages else 0, msg)
                token_count += msg_tokens
            else:
                break
        
        metadata = {
            'optimization': 'token_filtering',
            'original_count': len(messages),
            'optimized_count': len(optimized_messages),
            'original_tokens': current_tokens,
            'optimized_tokens': token_count,
            'compression_ratio': token_count / current_tokens if current_tokens > 0 else 0
        }
        
        print(f"🎯 최적화 완료: {len(messages)} -> {len(optimized_messages)} 메시지")
        print(f"📉 토큰 압축: {current_tokens} -> {token_count} ({metadata['compression_ratio']:.1%})")
        
        return optimized_messages, metadata
    
    def _estimate_tokens(self, text: str) -> int:
        """토큰 수 추정"""
        try:
            encoding = tiktoken.encoding_for_model("gpt-4")
            return len(encoding.encode(text))
        except:
            return len(text) // 4

    def create_conversation_summary(self, messages: List[Dict]) -> str:
        """대화 요약 생성"""
        try:
            conversation_text = "\n".join([
                f"{msg['role']}: {msg['content'][:200]}..."
                for msg in messages if msg['role'] in ['user', 'assistant']
            ])
            
            summary_prompt = f"""다음 대화를 간결하게 요약해주세요:

{conversation_text}

요약:"""
            
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "대화 내용을 간결하게 요약하세요."},
                    {"role": "user", "content": summary_prompt}
                ],
                max_tokens=300,
                temperature=0.3
            )
            
            return response.choices[0].message.content
        
        except Exception as e:
            logger.error(f"대화 요약 실패: {e}")
            return "이전 대화 요약을 생성할 수 없습니다."

# 컨텍스트 최적화 테스트
optimizer = ContextWindowOptimizer(max_tokens=1000)

# 테스트 메시지 생성
test_messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "안녕하세요! AI에 대해 설명해주세요." * 50},  # 긴 메시지
    {"role": "assistant", "content": "안녕하세요! AI는 인공지능을 의미합니다." * 30},
    {"role": "user", "content": "더 자세히 알고 싶습니다."},
    {"role": "assistant", "content": "물론입니다! 더 자세히 설명해드리겠습니다."},
]

total_tokens = sum(optimizer._estimate_tokens(msg['content']) for msg in test_messages)
optimized_messages, opt_metadata = optimizer.optimize_messages(test_messages, total_tokens)

print("\n📋 최적화 결과:")
for key, value in opt_metadata.items():
    print(f"  {key}: {value}")

## 5. 의도 분류 및 대화 상태 관리

사용자 의도를 분류하고 대화 상태를 추적하는 시스템을 구현합니다.

In [ ]:
class IntentClassifier:
    """의도 분류기"""
    
    def __init__(self):
        self.client = OpenAI(api_key=config.llm.openai_api_key)
        
        # 의도별 키워드 정의
        self.intent_keywords = {
            ConversationState.GREETING: ['안녕', '반갑', '처음', 'hello', 'hi'],
            ConversationState.INFORMATION_SEEKING: ['알려', '설명', '뭐야', '무엇', '어떻게', '왜'],
            ConversationState.TASK_EXECUTION: ['해줘', '만들어', '생성', '작성', '실행'],
            ConversationState.PROBLEM_SOLVING: ['문제', '오류', '에러', '안돼', '도움'],
            ConversationState.ENDING: ['고마워', '감사', '잘가', '안녕히', 'bye'],
            ConversationState.CASUAL_CHAT: ['재미있', '웃긴', '농담', '이야기']
        }
        
        print("✅ 의도 분류기 초기화 완료")
    
    def classify_intent(self, message: str) -> ConversationState:
        """메시지의 의도 분류"""
        print(f"🎯 의도 분류: {message[:30]}...")
        
        # 1단계: 키워드 기반 빠른 분류
        quick_intent = self._classify_by_keywords(message)
        if quick_intent != ConversationState.UNKNOWN:
            print(f"  ⚡ 키워드 매칭: {quick_intent.value}")
            return quick_intent
        
        # 2단계: LLM 기반 정확한 분류
        return self._classify_by_llm(message)
    
    def _classify_by_keywords(self, message: str) -> ConversationState:
        """키워드 기반 빠른 의도 분류"""
        message_lower = message.lower()
        
        for intent, keywords in self.intent_keywords.items():
            if any(keyword in message_lower for keyword in keywords):
                return intent
        
        return ConversationState.UNKNOWN
    
    def _classify_by_llm(self, message: str) -> ConversationState:
        """LLM 기반 정확한 의도 분류"""
        try:
            prompt = f"""다음 메시지의 의도를 분류하세요.

가능한 의도:
- greeting: 인사, 첫 만남
- information_seeking: 정보 요청, 질문
- task_execution: 작업 요청, 실행 명령
- problem_solving: 문제 해결, 도움 요청
- casual_chat: 일상 대화, 잡담
- ending: 대화 종료, 작별인사

메시지: {message}

의도 (위 카테고리 중 하나만):"""
            
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "대화 의도를 정확하게 분류하세요."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=20,
                temperature=0.1
            )
            
            result = response.choices[0].message.content.strip().lower()
            
            # 결과를 ConversationState로 변환
            intent_mapping = {
                'greeting': ConversationState.GREETING,
                'information_seeking': ConversationState.INFORMATION_SEEKING,
                'task_execution': ConversationState.TASK_EXECUTION,
                'problem_solving': ConversationState.PROBLEM_SOLVING,
                'casual_chat': ConversationState.CASUAL_CHAT,
                'ending': ConversationState.ENDING
            }
            
            classified_intent = intent_mapping.get(result, ConversationState.UNKNOWN)
            print(f"  🤖 LLM 분류: {classified_intent.value}")
            
            return classified_intent
        
        except Exception as e:
            logger.error(f"LLM 의도 분류 실패: {e}")
            return ConversationState.UNKNOWN

# 의도 분류 테스트
classifier = IntentClassifier()

test_messages = [
    "안녕하세요! 처음 뵙겠습니다.",
    "Python으로 웹 크롤링하는 방법을 알려주세요.",
    "코드를 작성해주세요.",
    "오류가 발생했는데 도움이 필요합니다.",
    "오늘 날씨가 좋네요.",
    "감사합니다. 안녕히 계세요."
]

print("\n🧪 의도 분류 테스트:")
for i, msg in enumerate(test_messages, 1):
    intent = classifier.classify_intent(msg)
    print(f"{i}. '{msg[:30]}...' -> {intent.value}")
    print()

## 6. 통합 멀티턴 챗봇 구현

모든 컴포넌트를 통합한 완전한 멀티턴 대화 시스템을 구현합니다.

In [ ]:
class MultiTurnChatbot:
    """통합 멀티턴 대화 챗봇"""
    
    def __init__(self):
        self.session_manager = SessionManager()
        self.context_optimizer = ContextWindowOptimizer()
        self.intent_classifier = IntentClassifier()
        self.client = OpenAI(api_key=config.llm.openai_api_key)
        
        print("✅ 멀티턴 챗봇 초기화 완료")
    
    def start_conversation(self, user_id: str) -> str:
        """새 대화 시작"""
        session_id, session_data = self.session_manager.create_session(user_id)
        print(f"🆕 새 대화 시작: {session_id[:8]}... (사용자: {user_id})")
        return session_id
    
    def chat(self, session_id: str, user_message: str) -> Dict[str, Any]:
        """대화 처리"""
        start_time = time.time()
        print(f"\n💬 대화 처리 시작: {user_message[:50]}...")
        
        try:
            # 1. 세션 조회
            session_data = self.session_manager.get_session(session_id)
            if not session_data:
                return {'error': '세션을 찾을 수 없습니다'}
            
            # 2. 의도 분류
            intent = self.intent_classifier.classify_intent(user_message)
            
            # 3. 사용자 메시지 추가
            user_msg = {
                'id': str(uuid.uuid4()),
                'role': 'user',
                'content': user_message,
                'timestamp': datetime.now().isoformat(),
                'metadata': {'intent': intent.value}
            }
            
            session_data['messages'].append(user_msg)
            session_data['current_state'] = intent.value
            session_data['last_activity'] = datetime.now().isoformat()
            
            # 4. 컨텍스트 최적화
            messages_for_api = [{'role': msg['role'], 'content': msg['content']} 
                               for msg in session_data['messages']]
            
            total_tokens = sum(self.context_optimizer._estimate_tokens(msg['content']) 
                             for msg in messages_for_api)
            
            optimized_messages, opt_metadata = self.context_optimizer.optimize_messages(
                messages_for_api, total_tokens
            )
            
            # 5. 시스템 프롬프트 추가
            system_prompt = self._generate_system_prompt(intent)
            final_messages = [{'role': 'system', 'content': system_prompt}] + optimized_messages
            
            # 6. AI 응답 생성
            print(f"🤖 AI 응답 생성 중... (메시지 수: {len(final_messages)})")
            response = self.client.chat.completions.create(
                model=config.llm.openai_model,
                messages=final_messages,
                temperature=config.llm.temperature,
                max_tokens=config.llm.max_tokens
            )
            
            ai_response = response.choices[0].message.content
            tokens_used = response.usage.total_tokens
            
            # 7. AI 응답 저장
            ai_msg = {
                'id': str(uuid.uuid4()),
                'role': 'assistant',
                'content': ai_response,
                'timestamp': datetime.now().isoformat(),
                'metadata': {
                    'model': config.llm.openai_model,
                    'tokens': tokens_used,
                    'state': intent.value
                }
            }
            
            session_data['messages'].append(ai_msg)
            session_data['total_tokens'] += tokens_used
            
            # 8. 세션 업데이트
            self.session_manager._save_session_data(session_id, session_data)
            
            processing_time = time.time() - start_time
            
            # 결과 반환
            result = {
                'session_id': session_id,
                'response': ai_response,
                'intent': intent.value,
                'conversation_state': session_data['current_state'],
                'processing_time': processing_time,
                'tokens_used': tokens_used,
                'optimization': opt_metadata,
                'session_stats': {
                    'message_count': len(session_data['messages']),
                    'total_tokens': session_data['total_tokens'],
                    'session_age': (datetime.now() - datetime.fromisoformat(
                        session_data['created_at'])).total_seconds()
                }
            }
            
            print(f"✅ 대화 처리 완료 - 시간: {processing_time:.2f}초, 토큰: {tokens_used}")
            return result
            
        except Exception as e:
            logger.error(f"대화 처리 실패: {e}")
            return {'error': f'대화 처리 중 오류: {str(e)}'}
    
    def _generate_system_prompt(self, intent: ConversationState) -> str:
        """상태별 시스템 프롬프트 생성"""
        base_prompt = "당신은 도움이 되는 AI 어시스턴트입니다."
        
        state_prompts = {
            ConversationState.GREETING: f"{base_prompt} 사용자와 첫 만남이므로 친근하게 인사하세요.",
            ConversationState.INFORMATION_SEEKING: f"{base_prompt} 정확한 정보를 제공하세요.",
            ConversationState.TASK_EXECUTION: f"{base_prompt} 단계별로 명확한 안내를 제공하세요.",
            ConversationState.PROBLEM_SOLVING: f"{base_prompt} 문제를 체계적으로 해결해주세요.",
            ConversationState.CASUAL_CHAT: f"{base_prompt} 자연스럽고 친근한 대화를 하세요.",
            ConversationState.ENDING: f"{base_prompt} 따뜻한 작별인사를 해주세요."
        }
        
        return state_prompts.get(intent, base_prompt)
    
    def get_conversation_history(self, session_id: str) -> Dict[str, Any]:
        """대화 히스토리 조회"""
        session_data = self.session_manager.get_session(session_id)
        if not session_data:
            return {'error': '세션을 찾을 수 없습니다'}
        
        return {
            'session_id': session_id,
            'user_id': session_data['user_id'],
            'created_at': session_data['created_at'],
            'current_state': session_data['current_state'],
            'message_count': len(session_data['messages']),
            'total_tokens': session_data['total_tokens'],
            'messages': session_data['messages'][-10:],  # 최근 10개 메시지만
        }

print("✅ 멀티턴 챗봇 클래스 정의 완료")

## 7. 멀티턴 대화 시나리오 테스트

실제 대화 시나리오로 시스템을 테스트합니다.

In [ ]:
# 멀티턴 챗봇 인스턴스 생성
chatbot = MultiTurnChatbot()

# 대화 시나리오 테스트
print("🎬 멀티턴 대화 시나리오 테스트 시작\n")

# 새 대화 시작
session_id = chatbot.start_conversation("demo_user")

# 대화 시나리오
conversation_script = [
    "안녕하세요! 처음 사용해봅니다.",
    "Python으로 웹 크롤링을 배우고 싶은데, 어디서 시작해야 할까요?",
    "requests와 BeautifulSoup을 사용한 예제 코드를 작성해주세요.",
    "코드에서 오류가 발생했어요. 도움이 필요합니다.",
    "정말 감사합니다. 많은 도움이 되었어요!"
]

# 대화 진행
conversation_results = []
for i, user_msg in enumerate(conversation_script, 1):
    print(f"\n{'='*60}")
    print(f"턴 {i}: 사용자 메시지")
    print(f"{'='*60}")
    print(f"👤 사용자: {user_msg}")
    
    # 챗봇 응답 생성
    result = chatbot.chat(session_id, user_msg)
    
    if 'error' not in result:
        print(f"\n🤖 AI ({result['intent']}): {result['response'][:200]}...")
        
        # 처리 통계
        print(f"\n📊 처리 통계:")
        print(f"  ⏱️  처리 시간: {result['processing_time']:.2f}초")
        print(f"  🎯 토큰 사용: {result['tokens_used']}")
        print(f"  💬 메시지 수: {result['session_stats']['message_count']}")
        print(f"  🔄 최적화: {result['optimization']['optimization']}")
        
        if result['optimization']['optimization'] != 'none':
            print(f"  📉 압축률: {result['optimization'].get('compression_ratio', 0):.1%}")
        
        conversation_results.append(result)
    else:
        print(f"❌ 오류: {result['error']}")
        break

print(f"\n{'='*60}")
print("🎉 대화 시나리오 테스트 완료")
print(f"{'='*60}")

# 최종 세션 통계
if conversation_results:
    final_stats = conversation_results[-1]['session_stats']
    print(f"\n📈 최종 세션 통계:")
    print(f"  📝 총 메시지 수: {final_stats['message_count']}")
    print(f"  🎯 총 토큰 사용: {final_stats['total_tokens']}")
    print(f"  ⏰ 세션 지속 시간: {final_stats['session_age']:.0f}초")
    print(f"  💰 평균 응답 시간: {sum(r['processing_time'] for r in conversation_results) / len(conversation_results):.2f}초")

# 대화 히스토리 조회
print(f"\n📚 대화 히스토리:")
history = chatbot.get_conversation_history(session_id)
if 'error' not in history:
    print(f"  세션 ID: {history['session_id'][:8]}...")
    print(f"  사용자 ID: {history['user_id']}")
    print(f"  현재 상태: {history['current_state']}")
    print(f"  메시지 수: {history['message_count']}")
else:
    print(f"  ❌ 히스토리 조회 실패: {history['error']}")

## 8. 성능 최적화 및 모니터링

대화 시스템의 성능을 분석하고 최적화 포인트를 확인합니다.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def analyze_conversation_performance(results: List[Dict]) -> Dict[str, Any]:
    """대화 성능 분석"""
    if not results:
        return {}
    
    # 데이터 추출
    processing_times = [r['processing_time'] for r in results]
    token_usage = [r['tokens_used'] for r in results]
    message_counts = [r['session_stats']['message_count'] for r in results]
    intents = [r['intent'] for r in results]
    
    # 통계 계산
    analysis = {
        'performance_stats': {
            'avg_processing_time': sum(processing_times) / len(processing_times),
            'max_processing_time': max(processing_times),
            'min_processing_time': min(processing_times),
            'total_tokens': sum(token_usage),
            'avg_tokens_per_turn': sum(token_usage) / len(token_usage),
            'max_tokens_per_turn': max(token_usage)
        },
        'optimization_stats': {
            'optimizations_applied': sum(1 for r in results if r['optimization']['optimization'] != 'none'),
            'avg_compression_ratio': sum(
                r['optimization'].get('compression_ratio', 0) 
                for r in results if r['optimization']['optimization'] != 'none'
            ) / max(1, sum(1 for r in results if r['optimization']['optimization'] != 'none'))
        },
        'intent_distribution': {}
    }
    
    # 의도 분포 계산
    from collections import Counter
    intent_counts = Counter(intents)
    analysis['intent_distribution'] = dict(intent_counts)
    
    return analysis

def visualize_performance(results: List[Dict]):
    """성능 시각화"""
    if not results:
        print("시각화할 데이터가 없습니다.")
        return
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('멀티턴 대화 성능 분석', fontsize=16)
    
    # 1. 처리 시간 추이
    turns = range(1, len(results) + 1)
    processing_times = [r['processing_time'] for r in results]
    ax1.plot(turns, processing_times, 'b-o', linewidth=2, markersize=6)
    ax1.set_title('턴별 처리 시간')
    ax1.set_xlabel('대화 턴')
    ax1.set_ylabel('처리 시간 (초)')
    ax1.grid(True, alpha=0.3)
    
    # 2. 토큰 사용량 추이
    token_usage = [r['tokens_used'] for r in results]
    ax2.bar(turns, token_usage, color='green', alpha=0.7)
    ax2.set_title('턴별 토큰 사용량')
    ax2.set_xlabel('대화 턴')
    ax2.set_ylabel('토큰 수')
    ax2.grid(True, alpha=0.3)
    
    # 3. 누적 메시지 수
    message_counts = [r['session_stats']['message_count'] for r in results]
    ax3.plot(turns, message_counts, 'r-s', linewidth=2, markersize=6)
    ax3.set_title('누적 메시지 수')
    ax3.set_xlabel('대화 턴')
    ax3.set_ylabel('메시지 수')
    ax3.grid(True, alpha=0.3)
    
    # 4. 의도 분포
    from collections import Counter
    intents = [r['intent'] for r in results]
    intent_counts = Counter(intents)
    ax4.pie(intent_counts.values(), labels=intent_counts.keys(), autopct='%1.1f%%')
    ax4.set_title('의도 분포')
    
    plt.tight_layout()
    plt.show()

# 성능 분석 실행
if 'conversation_results' in locals() and conversation_results:
    print("📊 성능 분석 시작...\n")
    
    analysis = analyze_conversation_performance(conversation_results)
    
    print("🎯 성능 통계:")
    perf_stats = analysis['performance_stats']
    print(f"  평균 처리 시간: {perf_stats['avg_processing_time']:.2f}초")
    print(f"  최대 처리 시간: {perf_stats['max_processing_time']:.2f}초")
    print(f"  총 토큰 사용: {perf_stats['total_tokens']}")
    print(f"  턴당 평균 토큰: {perf_stats['avg_tokens_per_turn']:.0f}")
    
    print("\n🔧 최적화 통계:")
    opt_stats = analysis['optimization_stats']
    print(f"  최적화 적용 횟수: {opt_stats['optimizations_applied']}")
    if opt_stats['avg_compression_ratio'] > 0:
        print(f"  평균 압축률: {opt_stats['avg_compression_ratio']:.1%}")
    
    print("\n🎭 의도 분포:")
    for intent, count in analysis['intent_distribution'].items():
        print(f"  {intent}: {count}회")
    
    # 성능 시각화
    print("\n📈 성능 차트 생성 중...")
    try:
        visualize_performance(conversation_results)
    except Exception as e:
        print(f"차트 생성 실패: {e}")
        print("Matplotlib 설치가 필요할 수 있습니다: pip install matplotlib")
else:
    print("분석할 대화 데이터가 없습니다. 먼저 대화 시나리오를 실행해주세요.")

## 9. 멀티 사용자 동시성 테스트

여러 사용자가 동시에 대화할 때의 시스템 안정성을 테스트합니다.

In [ ]:
import asyncio
import concurrent.futures
import threading
from threading import Thread
import time

def simulate_user_conversation(chatbot: MultiTurnChatbot, user_id: str, messages: List[str]) -> Dict[str, Any]:
    """사용자 대화 시뮬레이션"""
    print(f"👤 사용자 {user_id} 대화 시작")
    
    try:
        # 새 세션 시작
        session_id = chatbot.start_conversation(user_id)
        
        results = []
        total_time = 0
        
        for i, message in enumerate(messages):
            print(f"  {user_id}: {message[:30]}...")
            
            result = chatbot.chat(session_id, message)
            
            if 'error' not in result:
                results.append(result)
                total_time += result['processing_time']
                print(f"  ✅ 응답 완료 ({result['processing_time']:.2f}초)")
            else:
                print(f"  ❌ 오류: {result['error']}")
                break
            
            # 사용자 간 메시지 간격 시뮬레이션
            time.sleep(0.5)
        
        return {
            'user_id': user_id,
            'session_id': session_id,
            'total_messages': len(results),
            'total_time': total_time,
            'avg_response_time': total_time / len(results) if results else 0,
            'results': results
        }
    
    except Exception as e:
        print(f"  ❌ 사용자 {user_id} 대화 실패: {e}")
        return {'user_id': user_id, 'error': str(e)}

def run_concurrent_conversations():
    """동시 대화 테스트 실행"""
    print("🔄 멀티 사용자 동시성 테스트 시작\n")
    
    # 사용자별 대화 시나리오
    user_scenarios = {
        'user_1': [
            "안녕하세요!",
            "Python 기초를 배우고 싶어요.",
            "변수와 함수에 대해 설명해주세요."
        ],
        'user_2': [
            "반갑습니다.",
            "웹 개발을 시작하려면 어떻게 해야 하나요?",
            "HTML과 CSS부터 배워야 할까요?"
        ],
        'user_3': [
            "처음 사용해봅니다.",
            "데이터 분석에 관심이 있어요.",
            "pandas 라이브러리 사용법을 알려주세요."
        ]
    }
    
    # 동시성 테스트를 위한 스레드 풀
    start_time = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # 각 사용자별로 병렬 대화 시작
        future_to_user = {
            executor.submit(
                simulate_user_conversation, 
                chatbot, 
                user_id, 
                messages
            ): user_id
            for user_id, messages in user_scenarios.items()
        }
        
        # 결과 수집
        concurrent_results = []
        for future in concurrent.futures.as_completed(future_to_user):
            user_id = future_to_user[future]
            try:
                result = future.result()
                concurrent_results.append(result)
                print(f"✅ {user_id} 완료")
            except Exception as exc:
                print(f"❌ {user_id} 실패: {exc}")
    
    total_test_time = time.time() - start_time
    
    # 결과 분석
    print(f"\n{'='*50}")
    print("🎉 동시성 테스트 완료")
    print(f"{'='*50}")
    print(f"📊 전체 테스트 시간: {total_test_time:.2f}초")
    print(f"👥 동시 사용자 수: {len(user_scenarios)}")
    
    successful_users = [r for r in concurrent_results if 'error' not in r]
    
    if successful_users:
        print(f"✅ 성공한 사용자: {len(successful_users)}/{len(user_scenarios)}")
        
        avg_response_times = [r['avg_response_time'] for r in successful_users]
        total_messages = sum(r['total_messages'] for r in successful_users)
        
        print(f"\n📈 성능 메트릭:")
        print(f"  총 처리된 메시지: {total_messages}개")
        print(f"  평균 응답 시간: {sum(avg_response_times) / len(avg_response_times):.2f}초")
        print(f"  메시지 처리율: {total_messages / total_test_time:.1f} 메시지/초")
        
        # 사용자별 상세 결과
        print(f"\n👤 사용자별 결과:")
        for result in successful_users:
            print(f"  {result['user_id']}: {result['total_messages']}메시지, "
                  f"평균 {result['avg_response_time']:.2f}초")
    else:
        print("❌ 모든 사용자 대화가 실패했습니다.")
    
    return concurrent_results

# 동시성 테스트 실행
try:
    concurrent_test_results = run_concurrent_conversations()
    
    # Redis 연결 상태 확인
    print(f"\n🔗 시스템 상태:")
    print(f"  Redis 연결: {'활성' if chatbot.session_manager.client else '메모리 모드'}")
    print(f"  세션 관리: {'분산 저장' if chatbot.session_manager.client else '로컬 메모리'}")
    
except Exception as e:
    print(f"❌ 동시성 테스트 실패: {e}")
    print("일부 환경에서는 멀티스레딩 제한이 있을 수 있습니다.")

## 10. 실습 정리 및 확장 아이디어

4차시 실습의 핵심 내용을 정리하고 확장 가능한 아이디어를 제시합니다.

In [ ]:
def print_lesson4_summary():
    """4차시 실습 요약"""
    print("""\n📚 4차시 실습 요약: 대화 상태 관리 & 멀티턴 최적화
    
🎯 달성한 학습 목표:
✅ Redis 기반 세션 관리 시스템 구현
✅ 컨텍스트 윈도우 최적화 (토큰 기반 압축)
✅ 의도 분류 및 대화 상태 추적
✅ 멀티 사용자 동시성 처리
✅ 성능 모니터링 및 분석

🔧 핵심 구현 사항:
• SessionManager: Redis/메모리 하이브리드 세션 관리
• ContextWindowOptimizer: 중요도 기반 메시지 필터링 & 요약
• IntentClassifier: 키워드 + LLM 하이브리드 의도 분류
• MultiTurnChatbot: 통합 대화 관리 시스템

📊 성능 특징:
• 세션 기반 상태 관리로 일관된 대화 유지
• 토큰 최적화로 비용 효율성 확보
• 동시 사용자 지원으로 확장성 보장
• 실시간 성능 모니터링

🚀 확장 아이디어:
1. 고도화된 컨텍스트 관리
   - 벡터 기반 의미론적 유사성 계산
   - 장기/단기 메모리 분리
   - 사용자별 개인화 컨텍스트

2. 지능형 대화 흐름
   - 감정 분석 기반 톤 조절
   - 목표 지향적 대화 가이드
   - 다단계 태스크 관리

3. 고급 최적화
   - 적응형 토큰 할당
   - 예측적 컨텍스트 로딩
   - 모델별 최적화 전략

4. 운영 모니터링
   - 실시간 대시보드
   - 이상 상황 알림
   - A/B 테스트 프레임워크

5. 멀티모달 확장
   - 음성 대화 상태 관리
   - 이미지 기반 컨텍스트
   - 크로스 모달리티 세션""")

def suggest_next_steps():
    """다음 단계 제안"""
    print("""\n🎓 다음 학습 단계 제안:

📖 심화 학습:
• LangChain Memory 모듈 활용
• Semantic Kernel의 대화 관리
• 분산 시스템에서의 세션 관리
• 대화 품질 평가 메트릭

🛠️ 실전 프로젝트:
• 고객 서비스 챗봇 (장기 컨텍스트)
• 교육용 튜터 시스템 (개인화 학습 추적)
• 게임 NPC (상태 기반 대화)
• 상담 봇 (감정 상태 관리)

🔗 연관 기술:
• Vector databases (Pinecone, Weaviate)
• Stream processing (Kafka, Redis Streams)
• Monitoring tools (Prometheus, Grafana)
• A/B testing platforms""")

# 실습 요약 출력
print_lesson4_summary()
suggest_next_steps()

# 실습 완료 확인
print("\n" + "="*60)
print("🎉 4차시 실습이 성공적으로 완료되었습니다!")
print("📝 다음은 5차시: 외부 연동 & Tool Calling입니다.")
print("="*60)

# 실습 체크리스트
checklist = [
    "✅ Redis 기반 세션 관리 구현",
    "✅ 컨텍스트 윈도우 최적화 적용", 
    "✅ 의도 분류 시스템 동작",
    "✅ 멀티턴 대화 시나리오 테스트",
    "✅ 동시 사용자 처리 확인",
    "✅ 성능 메트릭 수집 및 분석"
]

print("\n📋 실습 체크리스트:")
for item in checklist:
    print(f"  {item}")

print("\n🔜 다음 실습 예고:")
print("  • OpenAI Function Calling")
print("  • 외부 API 연동 (날씨, 검색 등)")
print("  • 데이터베이스 연동")
print("  • Tool 기반 에이전트 구현")